In [39]:
import inspect
from functools import wraps

def memoize_components(static_method = True):
    """ A decorator for memoizing the components of an influence matrix or other method with components as the first arg

        

        Notes
        -----
        The cache is a dict with the same keys as previously passed components, when any of the other input arguments change the cache is deleted
        """
    if not isinstance(static_method, bool):
        raise ValueError('Memoise components is a decorator factory, it cannot be appled as a decorator directly. static_method argument must be a bool')
    
    def outer(fn):
        
        spec = [None]
        memo = dict()
        sig = inspect.signature(fn)
        if static_method:
            @wraps(fn)
            def inner(component, *args, **kwargs):
                nonlocal memo, spec, sig
                new_spec = sig.bind(None, *args, **kwargs)
                new_spec.apply_defaults()
                if not new_spec == spec[0]:
                    memo.clear()
                    del spec[0]
                    spec.append(new_spec)
                if component not in memo:
                    memo[component] = fn(component, *args, **kwargs)
                return memo[component]
        else:
            @wraps(fn)
            def inner(self, component, *args, **kwargs):
                nonlocal memo, spec, sig
                new_spec = sig.bind(None, None, *args, **kwargs)
                new_spec.apply_defaults()
                if not new_spec == spec[0]:
                    memo.clear()
                    del spec[0]
                    spec.append(new_spec)
                if component not in memo:
                    memo[component] = fn(self, component, *args, **kwargs)
                return memo[component]
        
        inner.memo = memo
        inner.spec = spec
        
        return inner
    return outer

class MyClass:
    def __init__(self, string):
        self.string = string
    
    @staticmethod
    @memoize_components(True)
    def find_component(component, a=1, b=2):
        print('finding component ...')
        return [component, a, b]
    
    
    @memoize_components(False)
    def fc(self, component, a=1, b=2):
        print('finding component ...')
        return [component, a, b]

a = MyClass('string')
c1 = a.find_component('xx')
b1 = a.fc('xx')

overwriting spec
finding component ...
overwriting spec
finding component ...


In [42]:
a.fc('xx',4)

overwriting spec
finding component ...


['xx', 4, 2]

In [24]:
c3 = a.find_component('xx', 10)
c2 is c3

<BoundArguments (component=None, a=None, b=10)>


True

In [15]:
c2

['stringxx', 1, 2]